In [ ]:
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage import io
from skimage.transform import resize
import os
from skimage.feature import hog
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from numpy import linalg as la
from skimage import io
from skimage.transform import resize
import os

from sklearn.metrics import confusion_matrix
import cv2
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


# Task 1

In [ ]:

x = []
y = []
for i in range(1,100):
    x.append(resize(cv2.imread('/Users/ayeshamoqeet/Desktop/ML/training_set/cats/cat.{}.jpg'.format(i)), (64, 32)))
    y.append(0)
for i in range(1,100):
    x.append(resize(cv2.imread('/Users/ayeshamoqeet/Desktop/ML/training_set/dogs/dog.{}.jpg'.format(i)), (64, 32)))
    y.append(1)

x, y = np.asarray(x), np.asarray(y)

x_train = []
for i in range(198):
    fd, hog_image = hog(x[i], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    x_train.append(fd)
x_train = np.asarray(x_train)


x_t = []
y_t = []
for i in range(1,100):
    x_t.append(resize(io.imread('/Users/ayeshamoqeet/Desktop/ML/test_set/cats/cat.{}.jpg'.format(4000+i)), (64, 32)))
    y_t.append(0)
for i in range(1,100):
    x_t.append(resize(io.imread('/Users/ayeshamoqeet/Desktop/ML/test_set/dogs/dog.{}.jpg'.format(4000+i)), (64, 32)))
    y_t.append(1)
y_t = np.array(y_t)

x_test = []
for i in range(198):
    fdt, hog_image = hog(x_t[i], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    x_test.append(fdt)
x_test = np.asarray(x_test)

def Predict_without_tiebreaker(x_test,k,n):
    predicted_results = []
    for j in range(0,len(x_test)):
        euclidean_distances = []
        for i in range(0, len(x_train)):
            euclidean_distances.append(la.norm((x_train[i] - x_test[j]), ord = n))
            min_indexs = np.argsort(euclidean_distances)[:k]
            y_ = y[min_indexs]
            counts = np.bincount(y_)
        predicted_results.append(np.argmax(counts))
    return predicted_results

def Predict(x_test,k,n):
    predicted_results = []
    for j in range(0,len(x_test)):
        euclidean_distances = []
        for i in range(0, len(x_train)):
            euclidean_distances.append(la.norm((x_train[i] - x_test[j]), ord = n))
            if k % 2 == 0:
                min_indexs = np.argsort(euclidean_distances)[:k-1]
            else:
                min_indexs = np.argsort(euclidean_distances)[:k]
            y_ = y[min_indexs]
            counts = np.bincount(y_)
        predicted_results.append(np.argmax(counts))
    return predicted_results



def f1_score(y_true, y_pred):
    tp = (y_true * y_pred).sum()
    fp = ((1 - y_true) * y_pred).sum()
    fn = (y_true * (1 - y_pred)).sum()
    precision = tp / (tp + fp )
    recall = tp / (tp + fn )
    f1 =  2 * (precision * recall) / (precision + recall )
    return f1



def knn(n):
    k_range = range(1, 8)
    scores = {}
    scores_list = []
    confusion_matrix = {}
    for k in k_range:
        y_pred = Predict(x_test,k,n)
        y_pred = np.array(y_pred)
        scores[k] = f1_score(y_t, y_pred)
        scores_list.append(f1_score(y_t, y_pred))
        confusion_matrix[k] = pd.crosstab(pd.Series((y_t), name='Actual'), pd.Series(y_pred, name='Predicted'))
    return scores_list, confusion_matrix


Euclidean_f1, CM = knn(2)
print(Euclidean_f1, CM)

Manhattan_f1, CM2= knn(1)
print(Manhattan_f1, CM2)


plt.figure(figsize=(12, 6))
plt.plot(range(1, 8),Euclidean_f1 , color='red')
plt.plot(range(1, 8),Manhattan_f1 , color='green', linestyle='dashed')

plt.title('F1_score vs K')
plt.xlabel('K Value')
plt.ylabel('F1 score')
plt.show()

# Task 2

In [ ]:
x = []
y = []

for i in range(1,4001):
    x.append(resize(cv2.imread('/Users/ayeshamoqeet/Desktop/ML/training_set/cats/cat.{}.jpg'.format(i)), (64, 32)))
    y.append(0)
    x.append(resize(cv2.imread('/Users/ayeshamoqeet/Desktop/ML/training_set/dogs/dog.{}.jpg'.format(i)), (64, 32)))
    y.append(1)

x, y = np.asarray(x), np.asarray(y)

x_train = []
for i in range(len(x)):
    fd, hog_image = hog(x[i], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    x_train.append(fd)
x_train = np.asarray(x_train)
print(x_train.shape)


x_t = []
y_t = []
for i in range(1,1001):
    x_t.append(resize(cv2.imread('/Users/ayeshamoqeet/Desktop/ML/test_set/cats/cat.{}.jpg'.format(4000+i)), (64, 32)))
    y_t.append(0)
    x_t.append(resize(cv2.imread('/Users/ayeshamoqeet/Desktop/ML/test_set/dogs/dog.{}.jpg'.format(4000+i)), (64, 32)))
    y_t.append(1)

x_test = []
for i in range(len(x_t)):
    fdt, hog_image = hog(x_t[i], orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    x_test.append(fdt)
x_test = np.asarray(x_test)
print(x_test.shape)


def knn(n):
    k_range = range(1, 8)
    scores = {}
    scores_list = []
    scores_acc = {}
    scores_acc_list = []
    confusion_matrix = {}
    for k in k_range:
        knn = KNeighborsClassifier(n_neighbors=k, metric=n)
        knn.fit(x_train, y)
        y_pred = knn.predict(x_test)
        scores[k] = f1_score(y_t, y_pred)
        scores_list.append(f1_score(y_t, y_pred))
        scores_acc[k] = metrics.accuracy_score(y_t, y_pred)
        scores_acc_list.append(metrics.accuracy_score(y_t, y_pred))
        confusion_matrix[k] = confusion_matrix(y_t, y_pred)

    return scores_list, confusion_matrix, scores_acc_list

Euclidean_f1, CM, Euclidean_acc  = knn('Euclidean')
print(Euclidean_f1, CM, Euclidean_acc)

Manhattan_f1, CM2, Manhattan_acc = knn('manhattan')
print(Manhattan_f1, CM2, Manhattan_acc)

plt.figure(figsize=(12, 6))
plt.plot(range(1, 8),Euclidean_f1 , color='red')
plt.plot(range(1, 8),Manhattan_f1 , color='green', linestyle='dashed')
plt.title('F1_score vs K')
plt.xlabel('K Value')
plt.ylabel('F1 score')
plt.show()

# Task 3

In [ ]:

TRAIN = '/Users/ayeshamoqeet/Desktop/ML/Training data'

data = []
Categories = ['Cloudy', 'Rain', 'Shine', 'Sunrise']
for category in Categories:
    folder = os.path.join(TRAIN, category)
    label = Categories.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_array = cv2.imread(img_path)
        data.append([img_array, label])
X = []
Y = []

for features, labels in data:
    X.append(features)
    Y.append(labels)

X = np.array(X, dtype=object)
Y = np.array(Y, dtype=int)


resized_img = []

for i in range(len(X)):
    resized_img.append(resize(X[i], (32, 32)))

resized_img = np.array(resized_img, dtype=object)
x_train = resized_img.flatten().reshape(899, 3072)


TEST = '/Users/ayeshamoqeet/Desktop/ML/Test data'
datat = []
Categoriest = ['Cloudy', 'Rain', 'Shine', 'Sunrise']
for category in Categoriest:
    folder = os.path.join(TEST, category)
    labelt = Categoriest.index(category)
    for img in os.listdir(folder):
        img_patht = os.path.join(folder, img)
        img_arrayt = cv2.imread(img_patht)
        datat.append([img_arrayt, labelt])
X_test = []
Y_test = []

for features, labels in datat:
    X_test.append(features)
    Y_test.append(labels)

X_test = np.array(X_test)  # , dtype = object)
Y_test = np.array(Y_test)

resized_imgt = []

for i in range(len(X_test)):
    resized_imgt.append(resize(X_test[i], (32, 32)))


resized_imgt = np.array(resized_imgt, dtype=object)
X_test = resized_imgt.flatten().reshape(224, 3072)
print(x_test.shape)



def Predict_without_tiebreaker(x_test,k,n):
    predicted_results = []
    for j in range(0,len(X_test)):
        euclidean_distances = []
        for i in range(0, len(x_train)):
            euclidean_distances.append(la.norm((x_train[i] - X_test[j]), ord = n))
            min_indexs = np.argsort(euclidean_distances)[:k]
            y_ = y[min_indexs]
            counts = np.bincount(y_)
        predicted_results.append(np.argmax(counts))
    return predicted_results

def Predict(x_test,k,n):
    predicted_results = []
    for j in range(0,len(X_test)):
        distances = []
        for i in range(0, len(x_train)):
            distances.append(la.norm((x_train[i] - X_test[j]), ord = n))
            if k % 2 == 0:
                min_indexs = np.argsort(distances)[:k-1]
            else:
                min_indexs = np.argsort(distances)[:k]
            y_ = Y_test[min_indexs]
            counts = np.bincount(y_)
        predicted_results.append(np.argmax(counts))
    return predicted_results



def f1_score(y_true, y_pred):
    tp = (y_true * y_pred).sum()
    fp = ((1 - y_true) * y_pred).sum()
    fn = (y_true * (1 - y_pred)).sum()
    precision = tp / (tp + fp )
    recall = tp / (tp + fn )
    f1 =  2 * (precision * recall) / (precision + recall )
    return f1


def knn(n):
    k_range = range(1, 8)
    scores = {}
    scores_list = []
    scores_acc = {}
    scores_acc_list = []
    confusion_matrix = {}
    for k in k_range:
        y_pred = Predict(X_test,k,n)
        y_pred = np.array(y_pred)
        scores[k] = f1_score((Y_test, y_pred), average='macro')
        scores_list.append(f1_score(Y_test, y_pred), average = 'macro')
        scores_acc[k] = metrics.accuracy_score(y_t, y_pred)
        scores_acc_list.append(metrics.accuracy_score(y_t, y_pred))
        confusion_matrix[k] = confusion_matrix(y_t, y_pred)
    return scores_list, confusion_matrix, scores_acc_list

Euclidean_f1, CM, Euclidean_acc = knn('Euclidean')
print(Euclidean_f1, CM, Euclidean_acc)

Manhattan_f1, CM2, Manhattan_acc = knn('manhattan')
print(Manhattan_f1, CM2, Manhattan_acc)

plt.figure(figsize=(12, 6))
plt.plot(range(1, 8), Euclidean_f1, color='red')
plt.plot(range(1, 8), Manhattan_f1, color='green', linestyle='dashed')
plt.title('F1_score vs K')
plt.xlabel('K Value')
plt.ylabel('F1 score')
plt.show()